A. Enfoque de validación
Utilicen la base respondieron. Para cada año, dividan las observaciones en
una base de prueba (test) y una de entrenamiento (train) utilizando el
comando train_test_split. La base de entrenamiento debe comprender el
70% de los datos, y la semilla a utilizar (random state instance) debe ser 444.
Establezca a desocupado como su variable dependiente en la base de
entrenamiento (vector y). El resto de las variables seleccionadas serán las
variables independientes (matriz X). Recuerden agregar la columna de unos
(1).
1. Cree una tabla de diferencia de medias entre la base de entrenamiento
y la de testeo de las características seleccionadas en su matriz X.
Comente la tabla de la diferencia de medias de sus variables entre
entrenamiento y testeo.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Cargar el archivo Excel
respondieron = pd.read_excel('respondieron.xlsx')

# Filtrar años 2004 y 2024
respondieron = respondieron[respondieron['AÑO'].isin([2004, 2024])]

# Crear la variable binaria 'desocupado'
respondieron['desocupado'] = (respondieron['ESTADO'] == 2).astype(int)

# Variables independientes
variables_x = ['P21', 'ingreso_ajustado', 'salario_semanal', 'edad', 'horastrab', 'edad2', 'educ']

# Recorrer por año
for anio in [2004, 2024]:
    datos = respondieron[respondieron['AÑO'] == anio].copy()
    
    # Crear X e y
    X = datos[variables_x].copy()
    X['constante'] = 1
    y = datos['desocupado']
    
    # Combinar X + y + año
    data = X.copy()
    data['desocupado'] = y
    data['AÑO'] = anio  # conservar la columna de año
    
    # Dividir en entrenamiento y prueba
    train, test = train_test_split(data, test_size=0.3, random_state=444)
    
    # Guardar a archivos Excel
    train.to_excel(f'base_entrenamiento_{anio}.xlsx', index=False)
    test.to_excel(f'base_prueba_{anio}.xlsx', index=False)



In [4]:
import pandas as pd

# Variables independientes (matriz X)
variables_x = ['P21', 'ingreso_ajustado', 'salario_semanal', 'edad',
               'horastrab', 'edad2', 'educ', 'constante']

# Función para calcular tabla de diferencia de medias entre dos archivos
def comparar_medias(archivo_2004, archivo_2024, variables):
    # Leer archivos
    df_2004 = pd.read_excel(archivo_2004)
    df_2024 = pd.read_excel(archivo_2024)

    # Calcular medias
    mean_2004 = df_2004[variables].mean()
    mean_2024 = df_2024[variables].mean()
    diff_abs = (mean_2004 - mean_2024).abs()

    # Tabla resumen
    tabla = pd.DataFrame({
        'Media 2004': mean_2004,
        'Media 2024': mean_2024,
        'Diferencia Absoluta': diff_abs
    })

    return tabla

# Comparar entrenamiento 2004 vs 2024
tabla_diff_train = comparar_medias('base_entrenamiento_2004.xlsx', 'base_entrenamiento_2024.xlsx', variables_x)
print("📊 Diferencias de medias entre bases de ENTRENAMIENTO (2004 vs 2024):")
print(tabla_diff_train, "\n")

# Comparar prueba 2004 vs 2024
tabla_diff_test = comparar_medias('base_prueba_2004.xlsx', 'base_prueba_2024.xlsx', variables_x)
print("📊 Diferencias de medias entre bases de PRUEBA (2004 vs 2024):")
print(tabla_diff_test)


📊 Diferencias de medias entre bases de ENTRENAMIENTO (2004 vs 2024):
                    Media 2004     Media 2024  Diferencia Absoluta
P21                 700.654373  352600.057790        351899.403417
ingreso_ajustado  36052.788988  352600.057790        316547.268802
salario_semanal     901.319725    8815.001445          7913.681720
edad                 38.169652      40.724646             2.554994
horastrab            35.879874      35.190935             0.688939
edad2              1600.306639    1806.294051           205.987412
educ                  5.384181       5.283333             0.100847
constante             1.000000       1.000000             0.000000 

📊 Diferencias de medias entre bases de PRUEBA (2004 vs 2024):
                    Media 2004     Media 2024  Diferencia Absoluta
P21                 814.776961  339880.916777        339066.139816
ingreso_ajustado  41925.067438  339880.916777        297955.849339
salario_semanal    1048.126686    8497.022919          7448.896

Como podemos observar los datos entre la base de entrenamiento de 2004 y la de prueba del mismo año son levemente distintas entre sí en los ingresos habituales (P21), por lo tanto los ingresos ajustados y el salario semanal van a ser distintos en la misma escala a los valores de 2024. En cambio P21, cambia muy poco en la base de entrenamiento de 2024 con la de prueba de 2024 por lo que podemos afirmar que por los casos tomados en la semilla 444 la varianza en el 2004 es mayor a la de 2024 en el ingreso. Esto en muy pequeña medida sugiere mayor igualdad de ingresos. De todas formas los patrones son similares entre entrenamiento y prueba, más que nada en las otras variables no mencionadas. 
Entre el 2004 y el 2024 podemos observar claramente las consecuencias inflacionarias ya que el ingreso subió más de 40 veces su valor y el ingreso real cayó casi un 20% teniendo en cuenta la base de prueba (la más real). La edad media de trabajo aumentó mas de dos años. Si bien puede haber sido por muchas razones creemos que esto es principalmente debido al aumento de la inmigración interna de jóvenes del interior buscando nuevas oportunidades en Buenos Aires u otros lugares más desarrollados durante esas décadas. Las horas trabajadas disminuyeron en promedio más de una hora quizás debido a las nuevas tecnologías que ahorran tiempo. Por ejemplo un celular para comunicarse más rápido. La educación casi no varía realmente. 
Por último la variable constante o intercepto ayuda a los modelos a ser más flexibles y ajustarse mejor a los datos ya que no tiene que pasar por el punto de origen.


B. Para los ocupados de la EPH en su region seleccionada usando la base 
de entrenamiento, estime los siguientes modelos usando como 
variable dependiente salario_semanal (y) y como predictores las 
variables creadas en el TP3: 
1) = salario_semanal en edad 
2) = salario_semanal en edad y edad2 
3) = salario_semanal en edad, edad2 y educ 
4) = salario_semanal en edad, edad2, educ y mujer (donde es una dymmy que toma mujer=1 si CH04==2) 
5) = salario_semanal en edad, edad2, educ, mujer y dos variables que haya creado y limpiado en el TP2 o TP3 que usted crean son relevantes para predecir salarios semanales. 
Complete y comente brevemente la siguiente tabla, reportando los 
coeficientes (hasta 3 decimales luego de la coma) y desvío estandar (sd 
con 2 decimales despues de la coma) de cada coeficiente entre 
parentesis:

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

respondieron = pd.read_excel('respondieron.xlsx')

# FILTRAMOS AÑOS
respondieron = respondieron[respondieron['AÑO'].isin([2004, 2024])]

# CREAMOS VARIABLE'desocupado'
respondieron['desocupado'] = (respondieron['ESTADO'] == 2).astype(int)

# LISTAMOS LAS VARIABLES QUE QUEREMOS VER
variables_x = ['P21', 'ingreso_ajustado', 'salario_semanal', 'edad', 'horastrab', 'edad2', 'educ', 'CH04']

for anio in [2004, 2024]:
    datos = respondieron[respondieron['AÑO'] == anio].copy()
    
    # CREAMOS X CON VARIABLES Y "CONSTANTE"
    X = datos[variables_x].copy()
    X['constante'] = 1
    
    # DEFINIMOS LA VARIABLE INDEPENDIENTE (target)
    y = datos['desocupado']
    
    # COMBINAMOS TODO EN UN DATA FRAME PARA DIVIDIR
    data = X.copy()
    data['desocupado'] = y
    data['AÑO'] = anio
    
    # DIVIDIMOS EN ENTRENAMIENTO Y PRUEBA
    train, test = train_test_split(data, test_size=0.3, random_state=444)
    
    # GUARDAMOS LAS BASES ACTUALIZADAS
    
    train.to_excel(f'base_entrenamiento_{anio}.xlsx', index=False)
    test.to_excel(f'base_prueba_{anio}.xlsx', index=False)

In [10]:
ocupados_2004 = pd.read_excel('base_entrenamiento_2004.xlsx')
ocupados_2004['mujer'] = (ocupados_2004['CH04'] == 2).astype(int)

In [12]:
y = ocupados_2004['salario_semanal']

In [14]:
import statsmodels.api as sm
# MODELO i) salario_semanal ~ edad
X1 = sm.add_constant(ocupados_2004[['edad']])
modelo1 = sm.OLS(y, X1).fit()
print(modelo1.summary())

# MODELO ii) salario_semanal ~ edad + edad2
X2 = sm.add_constant(ocupados_2004[['edad', 'edad2']])
modelo2 = sm.OLS(y, X2).fit()
print(modelo2.summary())


# MODELO iii) salario_semanal ~ edad + edad2 + educ
vars_modelo3 = ['edad', 'edad2', 'educ']

#  ARMAMOS UN DATAFRAME CON LOS PREDICTORES Y VARIABLES
df_modelo3 = ocupados_2004[vars_modelo3 + ['salario_semanal']].copy()

# ELIMINAMOS LOS VACIOS
df_modelo3 = df_modelo3.replace([np.inf, -np.inf], np.nan).dropna()

# CREAMOS X e Y 
X3 = sm.add_constant(df_modelo3[vars_modelo3])
y3 = df_modelo3['salario_semanal']

modelo3 = sm.OLS(y3, X3).fit()
print(modelo3.summary())


# MODELO iv) salario_semanal ~ edad + edad2 + educ + mujer

vars_modelo4 = ['edad', 'edad2', 'educ', 'mujer']

#CREAMOS DATA FRAME SOLO CON LAS VARIABLES QUE UTILIZA EL MODELO
df_modelo4 = ocupados_2004[vars_modelo4 + ['salario_semanal']].copy()

# LIMPIAMOS VACIOS NUEVAMENTE
df_modelo4 = df_modelo4.replace([np.inf, -np.inf], np.nan)

# ELIMINAMOS FILAS
df_modelo4 = df_modelo4.dropna()

# DEFINIMOS X e Y 
X4 = sm.add_constant(df_modelo4[vars_modelo4])
y4 = df_modelo4['salario_semanal']

modelo4 = sm.OLS(y4, X4).fit()
print(modelo4.summary())



# Modelo v) salario_semanal ~ edad + edad2 + educ + mujer + 'horastrab' y 'ingreso_ajustado'

vars_modelo5 = ['edad', 'edad2', 'educ', 'mujer', 'horastrab', 'ingreso_ajustado']

# CREAMOS DATA FRAME CON LAS VARIABLES PREDICTORAS + VARIABLE OBJETIVO
df_modelo5 = ocupados_2004[vars_modelo5 + ['salario_semanal']].copy()

# LIMPIAMOS
df_modelo5 = df_modelo5.replace([np.inf, -np.inf], np.nan)

# ELIMINAMOS FILAS
df_modelo5 = df_modelo5.dropna()

# DEFINIMOS MATRIZ X y VECTOR Y
X5 = sm.add_constant(df_modelo5[vars_modelo5])
y5 = df_modelo5['salario_semanal']

modelo5 = sm.OLS(y5, X5).fit()

print(modelo5.summary())

                            OLS Regression Results                            
Dep. Variable:        salario_semanal   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     16.75
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           4.62e-05
Time:                        01:36:15   Log-Likelihood:                -7759.8
No. Observations:                 949   AIC:                         1.552e+04
Df Residuals:                     947   BIC:                         1.553e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        536.3395     93.455      5.739      0.0

In [16]:
def resumen_modelo(modelo):
    coef = modelo.params.round(3)
    sd = modelo.bse.round(2)
    resumen = []
    for c, s in zip(coef, sd):
        resumen.append(f"{c} ({s})")
    return resumen

print("Modelo i) Coeficientes (sd):", resumen_modelo(modelo1))
print("Modelo ii) Coeficientes (sd):", resumen_modelo(modelo2))
print("Modelo iii) Coeficientes (sd):", resumen_modelo(modelo3))
print("Modelo iv) Coeficientes (sd):", resumen_modelo(modelo4))
print("Modelo v) Coeficientes (sd):", resumen_modelo(modelo5))

Modelo i) Coeficientes (sd): ['536.34 (93.45)', '9.562 (2.34)']
Modelo ii) Coeficientes (sd): ['-1115.421 (257.78)', '99.745 (13.36)', '-1.119 (0.16)']
Modelo iii) Coeficientes (sd): ['-931.974 (256.2)', '85.553 (13.24)', '-0.994 (0.16)', '0.446 (2.46)']
Modelo iv) Coeficientes (sd): ['-928.914 (256.4)', '86.324 (13.31)', '-1.003 (0.16)', '0.52 (2.47)', '-35.067 (57.74)']
Modelo v) Coeficientes (sd): ['-0.0 (0.0)', '0.0 (0.0)', '-0.0 (0.0)', '0.0 (0.0)', '-0.0 (0.0)', '-0.0 (0.0)', '0.025 (0.0)']


In [18]:
import pandas as pd
#REALIZAMOS UN RESUMEN DE LOS MODELOS
def tabla_resumen_modelos(modelos, nombres_modelos):
    
    variables = set()
    for modelo in modelos:
        variables.update(modelo.params.index)
    variables = sorted(variables)

    # <- CREAMOS UN DATA FRAME VACIO CON LAS VARIABLES
    resumen_df = pd.DataFrame(index=variables)

    # LLENAMOS LA TABLA 
    for modelo, nombre in zip(modelos, nombres_modelos):
        coef = modelo.params.round(3)
        sd = modelo.bse.round(2)
        # Formatear coef (sd)
        resumen_df[nombre] = [f"{coef.get(var, float('nan'))} ({sd.get(var, float('nan'))})" for var in variables]

    return resumen_df

# LISTAMOS LOS MODELOS
modelos = [modelo1, modelo2, modelo3, modelo4, modelo5]
nombres = ['Modelo i', 'Modelo ii', 'Modelo iii', 'Modelo iv', 'Modelo v']

#
tabla = tabla_resumen_modelos(modelos, nombres)

# MOSTRAMOS LA TABLA

tabla


,Modelo i,Modelo ii,Modelo iii,Modelo iv,Modelo v
const,536.34 (93.45),-1115.421 (257.78),-931.974 (256.2),-928.914 (256.4),-0.0 (0.0)
edad,9.562 (2.34),99.745 (13.36),85.553 (13.24),86.324 (13.31),0.0 (0.0)
edad2,nan (nan),-1.119 (0.16),-0.994 (0.16),-1.003 (0.16),-0.0 (0.0)
educ,nan (nan),nan (nan),0.446 (2.46),0.52 (2.47),0.0 (0.0)
horastrab,nan (nan),nan (nan),nan (nan),nan (nan),-0.0 (0.0)
ingreso_ajustado,nan (nan),nan (nan),nan (nan),nan (nan),0.025 (0.0)
mujer,nan (nan),nan (nan),nan (nan),-35.067 (57.74),-0.0 (0.0)


3. Enfoque de Validación: Ahora para cada modelo estime el salario 
predicho de testeo (salario_semanal_test sombrerito) usando las 
observaciones separadas de testeo y los coeficientes estimados en el 
apartado anterior. Reporte y comente las siguientes métricas de testo 
para cada modelo:

In [21]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# CARGAMOS LA BASE DE PRUEBA (TEST)
ocupados_2004_test = pd.read_excel('base_prueba_2004.xlsx')

# CREAMOS LA VARIABLE 'mujer' COMO EN EL ENTRENAMIENTO
ocupados_2004_test['mujer'] = (ocupados_2004_test['CH04'] == 2).astype(int)

# DEFINIMOS LA VARIABLE OBJETIVO EN TEST
y_test = ocupados_2004_test['salario_semanal']

# FUNCION PARA CALCULAR PREDICCIONES Y METRICAS
def evaluar_modelo(modelo, X_test, y_true):
    # PREDICCIONES
    y_pred = modelo.predict(X_test)
    # METRICAS
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return y_pred, mse, mae, r2

# MODELO i) VARIABLES PARA TEST
X1_test = sm.add_constant(ocupados_2004_test[['edad']].dropna())
y1_test = y_test.loc[X1_test.index]

y1_pred, mse1, mae1, r2_1 = evaluar_modelo(modelo1, X1_test, y1_test)

# MODELO ii)
X2_test = sm.add_constant(ocupados_2004_test[['edad', 'edad2']].dropna())
y2_test = y_test.loc[X2_test.index]

y2_pred, mse2, mae2, r2_2 = evaluar_modelo(modelo2, X2_test, y2_test)

# MODELO iii)
vars_modelo3 = ['edad', 'edad2', 'educ']
X3_test = sm.add_constant(ocupados_2004_test[vars_modelo3].replace([np.inf, -np.inf], np.nan).dropna())
y3_test = y_test.loc[X3_test.index]

y3_pred, mse3, mae3, r2_3 = evaluar_modelo(modelo3, X3_test, y3_test)

# MODELO iv)
vars_modelo4 = ['edad', 'edad2', 'educ', 'mujer']
X4_test = sm.add_constant(ocupados_2004_test[vars_modelo4].replace([np.inf, -np.inf], np.nan).dropna())
y4_test = y_test.loc[X4_test.index]

y4_pred, mse4, mae4, r2_4 = evaluar_modelo(modelo4, X4_test, y4_test)

# MODELO v)
vars_modelo5 = ['edad', 'edad2', 'educ', 'mujer', 'horastrab', 'ingreso_ajustado']
X5_test = sm.add_constant(ocupados_2004_test[vars_modelo5].replace([np.inf, -np.inf], np.nan).dropna())
y5_test = y_test.loc[X5_test.index]

y5_pred, mse5, mae5, r2_5 = evaluar_modelo(modelo5, X5_test, y5_test)

# ARMAMOS TABLA RESUMEN DE METRICAS

metricas_df = pd.DataFrame({
    'Modelo': ['Modelo i', 'Modelo ii', 'Modelo iii', 'Modelo iv', 'Modelo v'],
    'MSE': [mse1, mse2, mse3, mse4, mse5],
    'MAE': [mae1, mae2, mae3, mae4, mae5],
    'R2': [r2_1, r2_2, r2_3, r2_4, r2_5]
})

#MOSTRAMOS
metricas_df

,Modelo,MSE,MAE,R2
0,Modelo i,1.355748e+06,7.476352e+02,0.007288
1,Modelo ii,1.295131e+06,7.198772e+02,0.051673
2,Modelo iii,8.236601e+05,5.927584e+02,0.074361
3,Modelo iv,8.263058e+05,5.925232e+02,0.071388
4,Modelo v,1.548055e-25,2.659150e-13,1.000000
